In [2]:
import numpy as np
import pandas as pd 
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image  
import  kerastuner as kt
from kerastuner.tuners.randomsearch import RandomSearch
import time
import IPython
from keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.xception import Xception

In [ ]:
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [ ]:
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# sess

In [3]:
train_gen = ImageDataGenerator(rescale=1/255,  shear_range = 0.2,
                                zoom_range = 0.2, horizontal_flip = True, vertical_flip=True,
                                rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.4,
                                samplewise_center=True, 
                                samplewise_std_normalization=True,validation_split=0.25)
train_img = train_gen.flow_from_directory(directory='training',
                                              target_size=[128,128], subset='training' )

Found 1194 images belonging to 3 classes.


In [ ]:
#train_img.filenames

In [4]:
#val_gen = ImageDataGenerator(rescale=1/255)
val_img = train_gen.flow_from_directory(directory='training', target_size=[128,128], subset='validation')

Found 397 images belonging to 3 classes.


In [5]:
test_gen = ImageDataGenerator(rescale=1/255)
test_df = pd.read_csv('Dataset\\test.csv')
test_img = test_gen.flow_from_dataframe(dataframe=test_df, directory='Dataset\\TestData\\test',
                                        x_col='Filename', batch_size=16,
                                        class_mode=None, target_size=[128,128],
                                       shuffle=False, seed=3)

Found 314 validated image filenames.


In [ ]:
#test_img.filenames

In [ ]:
test_df.head()

In [ ]:
#test_img.filenames

In [ ]:
def model_builder(hp):
    model = Sequential()
    model.add(Conv2D(filters=hp.Int("Filter_1", 16, 128, 16), kernel_size=3,activation='relu', input_shape=(128,128,3)))
    model.add(MaxPool2D(strides=hp.Choice("Max_1", [1,2])))
    model.add(Conv2D(filters=hp.Int("Filter_2", 16, 128, 16), kernel_size=3,activation='relu'))
    model.add(MaxPool2D(strides=hp.Choice("Max_2", [1,2])))
    model.add(Conv2D(filters=hp.Int("Filter_2", 16, 128, 16), kernel_size=3,activation='relu'))
    model.add(MaxPool2D(strides=hp.Choice("Max_2", [1,2])))
    model.add(Flatten())
    model.add(Dense(units=hp.Int('units_1', 16, 128, 16), activation='relu'))
    model.add(Dropout(hp.Choice('drop_unit', values=[0.5,0.7,0.9])))
    model.add(Dense(units=hp.Int('units_2', 16, 126, 16), activation='relu'))
    model.add(Dense(units=3, activation='sigmoid'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
log_dir = f"{int(time.time())}"
log_dir=log_dir[4:-1]
class ClearOutput(Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)
    
tuned_model = RandomSearch(model_builder, objective='val_loss', max_trials=5,
                           directory= os.path.normpath('C:/projj/'+log_dir))
tuned_model.search(train_img, epochs=5, validation_data=val_img,  callbacks=[ClearOutput()])
best_hps = tuned_model.get_best_hyperparameters()[0]
final_model = tuned_model.hypermodel.build(best_hps)


In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=20)
final_model.fit(train_img, steps_per_epoch=200, validation_data = val_img, validation_steps=30, epochs = 45, callbacks=[es])

In [ ]:
#tuned_model.results_summary()

In [ ]:
model_1 = Sequential()
model_1.add(Conv2D(filters=32, kernel_size=3,activation='relu', input_shape=(64,64,3)))
model_1.add(MaxPool2D(strides=2))
model_1.add(Conv2D(filters=32, kernel_size=3,activation='relu'))
model_1.add(MaxPool2D(strides=2))
model_1.add(Conv2D(filters=64, kernel_size=3,activation='relu'))
model_1.add(MaxPool2D(strides=2)) 
model_1.add(Flatten())
model_1.add(Dense(units=512, activation='relu'))
#model_1.add(Dense(units=32, activation='relu'))
model_1.add(Dropout(0.6))
model_1.add(Dense(units=3, activation='softmax'))
model_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', patience=30)

model_1.fit(train_img, epochs=100, validation_data=val_img , callbacks=[es])

In [6]:
tran = Xception(include_top = False, weights='imagenet', input_shape=(128,128,3))

In [ ]:
#for layer in vgg.layers:
tran.trainable = True

In [7]:
tran.trainable

True

In [ ]:
Flatten()(tran.output)

In [ ]:
x= Flatten()(tran.output)
x=Dense(256, activation='relu')(x)
x = Dense(3, activation='softmax')(x)
model = Model(inputs=tran.inputs, outputs=x)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 1e-5), 
              loss='categorical_crossentropy', metrics=['accuracy'])
#es = EarlyStopping(monitor='val_loss', mode='min', patience=25)
model.fit(train_img, epochs=20, validation_data=val_img, steps_per_epoch = 40)

##### Submission box

In [ ]:
y_pred = model.predict(test_img)
y_pred = np.argmax(y_pred, axis=1)
submission_df = pd.DataFrame({'Filename': test_df['Filename'],
                             'Category': y_pred})
submission_df.replace({0:'Adults', 1:'Teenagers', 2:'Toddler'}, inplace= True)

In [ ]:
submission_df.to_csv('submission.csv', index = False)

In [ ]:
img = image.load_img('Dataset\\Sample Data\\000023.jpg', target_size=[64,64])
img = image.img_to_array(img)
#img = np.expand_dims(img, axis = 0)
img = img.reshape(-1, 64, 64, 3)

In [ ]:
result = model_1.predict(img)


In [ ]:
result

In [ ]:
result = np.argmax(result, axis=1)
result

In [ ]:
val_img.class_indices

In [ ]:
val_img.index_generator.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(val_img.classes, result)

In [ ]:
y_pred = final_model.predict(test_img)

In [ ]:
y_pred